In [1]:
import importlib
import pandas as pd #type: ignore
import numpy as np #type: ignore
import scipy.stats as stats #type: ignore
import sys, os
import datetime as dt

import pricer #type: ignore
importlib.reload(pricer)

pd.options.display.float_format = '{:,.8f}'.format
pd.options.display.max_rows = 100
pd.options.display.max_columns = 50

In [2]:
coinbaseexchange_creds = {
        'apiKey' : "ADD YOURS HERE",
        'secret' : "ADD YOURS HERE",
        'password': "ADD YOURS HERE",
    }

### Order Details

> Size, direction, limt

In [3]:
benchmark = "binance"
symbol = "DYDX/USDT"
hedge_symbol = "USDC/USDT"
perpetual = symbol+":USDT"
direction = "sell"
order_size = 1250932.68
limit_price = 0.96288 #None
dealer_fees = 28 #in bps
multiplier = 1 if direction=="buy" else -1
usdt_usd_conversion = True

> Times

In [4]:
timezone = "UTC" # "US/Eastern"
order_start_time = "2024-11-02 00:34:00"
order_update_time = "2024-11-02 11:40:00"
order_update_time_adj = (pd.to_datetime(order_update_time)-dt.timedelta(minutes=1)).strftime("%Y-%m-%d %H:%M:%S")
order_end_time = "2024-11-03 12:54:00"
order_duration_h = (pd.to_datetime(order_end_time)-pd.to_datetime(order_start_time)).total_seconds()/3600
#order_duration_h = 24 # uncomment this if you want to manually set the duration

> extras

In [5]:
expected_fill_per_minute = round(order_size / (order_duration_h * 60),2)

### Fetching Data

In [10]:
# The pricer lib is a powerful class that I coded for the 1st version of the pricer.
# It solely focus on fetching market data. You write the exchange you want to fetch data, 
# the spot pair and perp pair (if you want to fetch for perp as well, otherwise you don't need it)
# The class will immediately fetch current snapshot of orderbook, ohlcv data for spot and perp (with the time range
# that you can modify or leave it as default, check the class for default values), spot and perp mid, top bid and ask for
# spot and perps, historical funding rate and predicted next funding rate
# You need to instantiate 1 class per exchange and per pair

# Below I'm instantiating the class for Binance and for SUI, and increasing the interval fetched for "1m" candles
# Numbers on modified_date_range are hours. i.e. for "1m" we are fetching 96h

#modified_date_range = {"1s": 24, "1m": 1, "5m": 1, "1h": 1, "1d": 24}
modified_date_range = {"1s":1, "1m": 96}
benchmark_market_data = pricer.MarketData(benchmark, symbol, date_range = modified_date_range)
#coinbase_market_data = pricer.MarketData(benchmark, hedge_symbol, date_range = modified_date_range)#, credentials=coinbaseexchange_creds)

ExchangeNotAvailable: binance GET https://api.binance.com/api/v3/exchangeInfo 451  {
  "code": 0,
  "msg": "Service unavailable from a restricted location according to 'b. Eligibility' in https://www.binance.com/en/terms. Please contact customer service if you believe you received this message in error."
}

In [11]:
# binance_usd = pd.merge_asof(benchmark_market_data.ohlcv_data["spot"]["1s"][["close"]], coinbase_market_data.ohlcv_data["spot"]["1m"][["close"]], on="datetime", direction="nearest", suffixes=("_binance", "_coinbase"))
# binance_usd["close_adjusted"] = binance_usd["close_binance"] * binance_usd["close_coinbase"]
# binance_usd.to_csv("binance_usd_1m.csv", index=False)

### Working the Data

In [8]:
# Copying the dataframe so we can modify it as we want without affecting the class
symbol_1m_ohlcv = benchmark_market_data.ohlcv_data["spot"]["1m"].copy()
usdt_1m_ohlcv = coinbase_market_data.ohlcv_data["spot"]["1m"].copy()
usd_hedge_quote_ccy = hedge_symbol.split("/")[1]

symbol_1m_ohlcv = symbol_1m_ohlcv.loc[order_start_time:order_update_time_adj]
usdt_1m_ohlcv = usdt_1m_ohlcv.loc[order_start_time:order_update_time_adj]

if limit_price is not None:
    symbol_1m_ohlcv["tradable"] = np.where(symbol_1m_ohlcv["close"]>=limit_price, True, False) if direction=="sell" else np.where(symbol_1m_ohlcv["close"]<=limit_price, True, False)
    symbol_1m_ohlcv["tradable_test"] = (symbol_1m_ohlcv["close"] >= limit_price) if direction == "sell" else (symbol_1m_ohlcv["close"] <= limit_price)
else:
    symbol_1m_ohlcv["tradable"] = True

symbol_close_tradable_df = symbol_1m_ohlcv.query("tradable == True")[["close"]].copy()
n_close_above_limit = len(symbol_close_tradable_df)
usdt_close_above_limit_df = usdt_1m_ohlcv.merge(symbol_close_tradable_df, left_index=True, right_index=True, suffixes=("", "_symbol"))[["close", "close_symbol"]]

NameError: name 'benchmark_market_data' is not defined

In [9]:
symbol_avg_price = round(symbol_close_tradable_df["close"].mean(),8)
usdt_avg_price = round(usdt_close_above_limit_df["close"].mean(),8)
final_price_usd_adjusted = round(symbol_avg_price * usdt_avg_price,8) if usd_hedge_quote_ccy == "USD" else round(symbol_avg_price / usdt_avg_price,8)
net_final_price_usdt = round(symbol_avg_price*(1+(multiplier*dealer_fees)/10000),8)
net_final_price_usd_adjusted = round(final_price_usd_adjusted*(1+(multiplier*dealer_fees)/10000),8)
print(f"Order Details\n\nSymbol: {symbol}\nDirection: {direction}\nSize: {order_size:,.8f}\nLimit Price: {limit_price}\nFrom: {order_start_time} To: {order_update_time} UTC\n")
print(f"{symbol} Average: {symbol_avg_price}\n{hedge_symbol} Average: {usdt_avg_price}\n\
{symbol}(C) Average: {final_price_usd_adjusted}\nNet Price USD Adjusted({dealer_fees}bps fees): {net_final_price_usd_adjusted}\nNet Price USDT ({dealer_fees}bps fees): {net_final_price_usdt}\nNº of tradable close prices: {n_close_above_limit}/{len(symbol_1m_ohlcv)}\nExpected Fills: {n_close_above_limit*expected_fill_per_minute:,.4f}")

NameError: name 'symbol_close_tradable_df' is not defined